In [4]:
from data_preparator import DataPreparator
test = DataPreparator.load_from_pickle('pickle/train_documents.pkl')

Object loaded from pickle/train_documents.pkl


In [18]:
from letor import LETOR

ranker_path = "lgbr_base.txt"
model_path = "lsi_base.model"
data_path = r'pickle\test_data.pkl'
qrel_path = r"qrels-folder\test_qrels.txt"
letor = LETOR(ranker_path, model_path, data_path)
qrels = letor.data_preparator.read_qrels_test(qrel_path)

File not exists at pickle\test_data.pkl


In [7]:
from letor import LETOR
import pandas as pd

ranker_path = "lgbr_base.txt"
model_path = "lsi_base.model"
data_path = r'pickle\test_data.pkl'
qrel_path = r"qrels-folder\test_qrels.txt"

letor = LETOR(ranker_path, model_path, data_path)
qrels = letor.data_preparator.read_qrels_test(qrel_path)

data = letor.data

Object loaded from pickle\test_data.pkl


In [43]:
ndcgs_letor = []
ndcgs_bsbi = []

columns = ['query_id', 'doc_id', 'score', 'relevance']
df_letor = pd.DataFrame(columns=columns)
df_bsbi = pd.DataFrame(columns=columns)

In [44]:
from tqdm import tqdm
# iterate untuk setiap query
for qid in tqdm(qrels):
    dataset, doc_ids, raw_scores = data[qid]
    qrel = qrels[qid]
    k = 25

    # melakukan preparation data test dan prediksi 
    X = letor.ranker.prepare_data_test(dataset)
    pred = letor.ranker.predict_ranker(X)
    
    # untuk mendapatkan top k dokumen
    doc_pred_pairs = list(zip(doc_ids, pred))
    sorted_pairs = sorted(doc_pred_pairs, key=lambda x: x[1], reverse=True)
    top_k_doc_ids_letor = [pair[0] for pair in sorted_pairs[:k]]
    top_k_doc_ids_letor = ['D'+ doc for doc in top_k_doc_ids_letor]

    top_k_doc_ids_bsbi = doc_ids[:k]
    top_k_doc_ids_bsbi = ['D'+ doc for doc in top_k_doc_ids_bsbi]

    # mendapatkan ranking dari top k doc yang di sort
    ranking_letor = [qrel[did] for did in top_k_doc_ids_letor]
    ranking_bsbi = [qrel[did] for did in top_k_doc_ids_bsbi]
    
    # Hasil nDCG untuk model "letor" dan model "bsbi" akan dihitung dan disimpan.
    ndcg_score_letor = LETOR.ndcg(ranking_letor)
    ndcg_score_bsbi = LETOR.ndcg(ranking_bsbi)

    ndcgs_letor.append(ndcg_score_letor)
    ndcgs_bsbi.append(ndcg_score_bsbi)
    
    # Create DataFrames for 'letor' and 'bsbi' scores in the current loop
    df_letor_loop = pd.DataFrame({'query_id': [qid]*len(ranking_letor),
                                'doc_id': top_k_doc_ids_letor,
                                'score': [pair[1] for pair in sorted_pairs[:k]],
                                'relevance': ranking_letor})
    
    df_bsbi_loop = pd.DataFrame({'query_id': [qid]*len(ranking_letor),
                                'doc_id': top_k_doc_ids_bsbi,
                                'score': raw_scores[:k],
                                'relevance': ranking_bsbi})

    # Concatenate the DataFrames to the main DataFrames
    df_letor = pd.concat([df_letor, df_letor_loop], ignore_index=True)
    df_bsbi = pd.concat([df_bsbi, df_bsbi_loop], ignore_index=True)

print("Mean NDCG Score for Letor:", sum(ndcgs_letor) / len(ndcgs_letor))
print("Mean NDCG Score for BSBI:", sum(ndcgs_bsbi) / len(ndcgs_bsbi))

  0%|          | 0/700 [00:00<?, ?it/s]

C:\Users\bryan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\spatial\distance.py:636: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
C:\Users\bryan\AppData\Local\Temp\ipykernel_26612\2218446069.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_letor = pd.concat([df_letor, df_letor_loop], ignore_index=True)
C:\Users\bryan\AppData\Local\Temp\ipykernel_26612\2218446069.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior

Mean NDCG Score for Letor: 0.45223863030279443
Mean NDCG Score for BSBI: 0.6134689162548457


In [45]:
# Save df_letor to a CSV file
df_letor.to_csv('df_letor.csv', index=False)

# Save df_bsbi to a CSV file
df_bsbi.to_csv('df_bsbi.csv', index=False)